In [26]:
pip install meta-ai-api openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests
import json
import re


In [50]:
openai_api_key = ""
openai_api_key_2 = ""
mistral_ai_key = ""
JAMBA_API_KEY = ''
JAMBA_API_URL = 'https://api.ai21.com/studio/v1/chat/completions'

In [67]:
from openai import OpenAI
openai_client = OpenAI(api_key=openai_api_key_2)


In [ ]:
from meta_ai_api import MetaAI # no api key needed :O

meta_ai = MetaAI()

/home/chris/Documents/UT/SWTest/project/QualitySoftware/venv/lib/python3.12/site-packages/requests/models.py:754: RuntimeWarning: coroutine 'ask_sydney_async' was never awaited
  @property


In [46]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [40]:
def send_message_to_jamba(message):
    headers = {
        'Authorization': f'Bearer {JAMBA_API_KEY}',
        'Content-Type': 'application/json'
    }
    data = {
        "model": "jamba-1.5-large",
        "messages": [
            {"role": "user", "content": message}
        ]
    }

    response = requests.post(JAMBA_API_URL, headers=headers, json=data)
    response.raise_for_status()  # Raises an error if the request failed
    return response.json()


In [45]:
def submit_prompt_openai(prompt: str, model: str = "gpt-4o-mini") -> str:
    """
    Submits a prompt to ChatGPT and returns the response message.
    
    Args:
        prompt (str): The prompt to submit.
        model (str): The model to use (default is "gpt-4o").

    Returns:
        str: The response message from ChatGPT.
    """
    try:
        completion = openai_client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": [{"type": "text", "text": prompt}]
                }
            ]
        )
        return completion.choices[0].message.content

    except Exception as e:
        print(f"An error occurred: {e}")
        return ""



In [74]:
import os
from mistralai import Mistral

model = "mistral-large-latest"

client = Mistral(api_key=mistral_ai_key)

chat_response = client.chat.complete(
    model = model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)

print(chat_response.choices[0].message.content)


Choosing the "best" French cheese can be subjective as it greatly depends on personal taste. France is renowned for its wide variety of cheeses, with over 400 different types. Here are a few highly regarded French cheeses across various categories:

1. **Soft Cheeses:**
   - **Brie de Meaux**: Known for its creamy texture and rich, slightly nutty flavor.
   - **Camembert de Normandie**: Soft and creamy with a distinctive earthy taste.

2. **Semi-Soft Cheeses:**
   - **Reblochon**: A nutty and fruity cheese from the Alps, often used in tartiflette.
   - **Morbier**: Recognizable by its layer of ash in the middle, it has a creamy texture and a fruity, slightly smoky flavor.

3. **Hard Cheeses:**
   - **Comté**: A firm, nutty, and sweet cheese made from unpasteurized cow's milk.
   - **Beaufort**: Similar to Comté but with a more pronounced, complex flavor.

4. **Blue Cheeses:**
   - **Roquefort**: A tangy and salty sheep milk cheese with distinctive veins of blue mold.
   - **Bleu d'Auve

In [ ]:
import asyncio
from sydney import SydneyClient # copilot at home

async def get_response(prompt: str) -> str:
    async with SydneyClient() as sydney:
        response = ""
        async for chunk in sydney.ask_stream(prompt):
            response += chunk
        return response

# Function to be used in Jupyter Notebook
async def ask_sydney(prompt: str):
    """goofy ah"""
    try:
        response = await get_response(prompt)
        return response
    except Exception as e:
        print(f"An error occurred: {e}")



In [32]:
message = await ask_sydney("What should I bring for my upcoming picnic?")
print(message)

A picnic 🧺 is a great way to enjoy nature and have some fun with your friends or family. You should bring along some sandwiches, fruits 🍎, drinks 🥤, snacks 🍪, games 🎮, and a blanket 🛏. What kind of picnic are you planning?


In [41]:
def extract_code_from_llm_response(llm_response: str) -> str:
    """try to parse the java program from the input llm response"""
    # look for key words denoting the start of a java program
    key_words = ["import", "package", "public class"]
    for key_word in key_words:
        start_index = llm_response.find(key_word)
        if (start_index != -1):
            break
    if (start_index == -1):
        print("ERROR: unable to find start of java program")
        return ""
    # look for the closing curly brace for the end of the java program
    end_index = llm_response.rfind("}") + 1
    if (end_index == -1):
        print("ERROR: unable to find end of java program")
        return ""
    # return the found program
    generated_code = llm_response[start_index:end_index]
    return generated_code


In [42]:
def get_class_name_from_generated_code(generated_code: str) -> str:
    match = re.search(r'public class (\w+)', generated_code)
    if (match):
        class_name = match.group(1)
        return class_name
    else:
        print("ERROR: class name not found in generated code")

In [43]:
def add_package_and_import_to_generated_code(package_name: str, class_under_test: str, generated_code: str) -> str:
    import_statement = f'import {class_under_test};'
    return f"package {package_name};\n" + f"{import_statement}\n\n" + generated_code


In [47]:
def generate_code(prompt: str, llm: str, package_name: str, class_name: str, write_output: bool = False) -> None:
    print("Generating code...")

    file_path = f"../prompts/{package_name}/{prompt}.txt"
    with open(file_path, 'r') as file:
        prompt_text = file.read()

    if (llm=="metaAI"):
        response = meta_ai.prompt(message=prompt_text)
        message = response["message"]
    elif (llm=="ai21Jamba"):
        response = send_message_to_jamba(prompt_text)
        message = response['choices'][0]['message']['content']
    elif (llm=="chatGPT4o"):
        message = submit_prompt_openai(prompt_text, model='gpt-4o')
    elif (llm=="chatGPTo1Preview"):
        print("Error, o1-preview not available via API :/")
    elif (llm=="copilot"):
        message = asyncio.run(ask_sydney(prompt_text))
    else: 
        print("Error, llm not supported, please pick a valid option")
        return

    print(message)

    generated_code = extract_code_from_llm_response(message)
    package = f"{package_name}.{llm}.{prompt}"
    class_under_test = f"{package_name}.{class_name}"
    generated_code = add_package_and_import_to_generated_code(package, class_under_test, generated_code)


    if (write_output):
        output_filename = f"../src/test/java/{package_name}/{llm}/{prompt}/{class_name}Test.java"
        with open(output_filename, 'w') as file:
            file.write(generated_code)
        print("Wrote generated code to ", output_filename)
    else:
        print(generated_code)

In [51]:
generate_code("prompt1", "copilot", "anagrams", "Anagrams", write_output=False)

Generating code...
Sure, I can write a test suite for you. Here is the test case: @Test public void test_grouping() { assertEquals(3, strs.length); // make sure there are three strings in the array String[] expected = {"a", "b", "c"}; // create a new array of arrays with three elements ArrayList<ArrayList<String>> result = new ArrayList<>(); // loop through each string and create a new array of arrays for (int i = 0; i < strs.length; i++) { // get the first element of the current string String current = strs[i]; // get the second element of the current string String second = strs[i + 1]; // add them to the current array of arrays ArrayList<String> temp = new ArrayList<>(); // add them to the temp array temp.add(current); temp.add(
ERROR: unable to find start of java program
package anagrams.copilot.prompt1;
import anagrams.Anagrams;


